<a href="https://colab.research.google.com/github/mariasta72/D-Evia_Info-Kit/blob/main/Progressive_Payment_Calculator_v26_06_25_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Default title text
import pandas as pd
import numpy as np

# --- Install numpy_financial if not already present ---
try:
    import numpy_financial as npf
except ImportError:
    print("numpy_financial not found. Installing...")
    !pip install numpy_financial
    import numpy_financial as npf
    print("numpy_financial installed successfully.")

# Ensure pandas display options are set for later DataFrame use (if needed for debugging)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)


def integrated_progressive_payment_calculator(total_spa_price, months_to_vp, loan_percentage, annual_interest_rate, loan_tenure_years):
    """
    Calculates and displays integrated monthly payments (pre-VP progressive interest + post-VP full amortization).
    """

    # --- Input Validation ---
    if not all(isinstance(arg, (int, float)) and arg >= 0 for arg in [total_spa_price, months_to_vp, loan_percentage, annual_interest_rate, loan_tenure_years]):
        print("Error: All inputs must be non-negative numbers.")
        return
    if loan_percentage > 100 or loan_percentage < 0: # Added < 0 check
        print("Error: Loan percentage must be between 0 and 100%.")
        return
    if loan_tenure_years <= 0:
        print("Error: Loan tenure must be greater than 0 years.")
        return
    if total_spa_price <= 0:
        print("Error: Total SPA Price must be greater than 0.")
        return
    if annual_interest_rate < 0: # Added < 0 check
        print("Error: Annual interest rate cannot be negative.")
        return

    # Convert percentages to decimals
    loan_percentage_decimal = loan_percentage / 100
    monthly_interest_rate = (annual_interest_rate / 100) / 12
    total_loan_amount = total_spa_price * loan_percentage_decimal
    total_loan_months = loan_tenure_years * 12

    print(f"\n--- Property & Loan Summary ---")
    print(f"Total SPA Price: RM {total_spa_price:,.2f}")
    print(f"Estimated Months to VP: {months_to_vp} months")
    print(f"Loan Percentage: {loan_percentage:.2f}%")
    print(f"Annual Interest Rate: {annual_interest_rate:.2f}%")
    print(f"Loan Tenure: {loan_tenure_years} years ({total_loan_months} months)")
    print(f"Total Loan Amount: RM {total_loan_amount:,.2f}")
    print("-" * 50)

    # Standard Schedule H Payment Milestones (as percentages of SPA price)
    # These are the *trigger points* for loan drawdown.
    schedule_h_milestones_triggers = {
        0: 10, # Month 0: Upon Signing SPA (e.g., 10% of SPA price)
        "Completion of Foundation": 10,
        "Completion of Structure": 15,
        "Completion of Walls": 10,
        "Completion of Roofing, Electrical, Plumbing": 10,
        "Completion of Internal & External Plastering": 10,
        "Completion of Sewerage Works": 5,
        "Completion of Drains": 5,
        "Completion of Roads": 5,
        "Vacant Possession (VP) with CCC": 12.5, # This happens at months_to_vp
        "Submission for Subdivision of Building": 2.5,
        "6 Months After VP (Defect Liability Period - 1st Half)": 2.5,
        "18 Months After VP (Defect Liability Period - 2nd Half)": 2.5
    }

    # Calculate sum of milestone percentages, ensuring 0 key is treated as a numeric value
    current_sum_percent = 0
    for k, v in schedule_h_milestones_triggers.items():
        current_sum_percent += v

    if current_sum_percent != 100:
        # Correcting the f-string for the warning message
        print(f"Warning: Schedule H milestone percentages sum to {current_sum_percent:.2f}%, not 100%. Adjusting last stage proportionally.")
        last_key = list(schedule_h_milestones_triggers.keys())[-1]
        schedule_h_milestones_triggers[last_key] += (100 - current_sum_percent)

    # --- Pre-VP Calculation: Progressive Interest Payments ---

    # Track loan drawdown at specific months
    loan_drawdowns = {} # Key: estimated_month, Value: loan_drawn_at_this_month

    cumulative_loan_drawn_at_milestone = 0
    estimated_month_at_milestone = 0

    # Determine milestones that actually involve construction progress leading to VP
    pre_vp_construction_milestones = [
        k for k in schedule_h_milestones_triggers.keys()
        if isinstance(k, str) and k not in ["Vacant Possession (VP) with CCC", "Submission for Subdivision of Building", "6 Months After VP (Defect Liability Period - 1st Half)", "18 Months After VP (Defect Liability Period - 2nd Half)"]
    ]

    # Handle "Upon Signing SPA" (Month 0) separately as it's not a named construction milestone in the list
    initial_spa_percent = schedule_h_milestones_triggers.get(0, 0) # Get 10% if present, else 0
    if initial_spa_percent > 0:
        spa_amount_initial = total_spa_price * (initial_spa_percent / 100)
        loan_drawn_initial = spa_amount_initial * loan_percentage_decimal

        if loan_drawn_initial > total_loan_amount:
            loan_drawn_initial = total_loan_amount

        loan_drawdowns[0] = loan_drawn_initial
        cumulative_loan_drawn_at_milestone = loan_drawn_initial

    # Distribute months_to_vp across the relevant pre-VP milestones for month estimation
    if len(pre_vp_construction_milestones) > 0 and months_to_vp > 0:
        # Divide months to VP by the number of construction stages (excluding the very first 0-month one)
        month_increment_per_stage = months_to_vp / len(pre_vp_construction_milestones)
    else:
        month_increment_per_stage = 0

    # Simulate loan drawdowns and estimate months for each stage
    for i, milestone_key in enumerate(pre_vp_construction_milestones):
        percentage = schedule_h_milestones_triggers[milestone_key]

        # Estimate month for this construction stage
        if months_to_vp > 0:
             # Progress the month based on the order of construction milestones
             estimated_month_at_milestone = round((i + 1) * month_increment_per_stage)
             if estimated_month_at_milestone > months_to_vp: # Cap at VP month
                 estimated_month_at_milestone = months_to_vp
        else:
            estimated_month_at_milestone = i + 1 # Just increment if VP is 0 months

        # Calculate loan drawn for this specific milestone
        spa_amount_this_stage = total_spa_price * (percentage / 100)
        loan_drawn_this_stage = spa_amount_this_stage * loan_percentage_decimal

        # Cap to total loan amount
        if cumulative_loan_drawn_at_milestone + loan_drawn_this_stage > total_loan_amount + 0.01:
            loan_drawn_this_stage = max(0, total_loan_amount - cumulative_loan_drawn_at_milestone)

        cumulative_loan_drawn_at_milestone += loan_drawn_this_stage

        # Store drawdown amount at the estimated month
        loan_drawdowns[estimated_month_at_milestone] = loan_drawdowns.get(estimated_month_at_milestone, 0) + loan_drawn_this_stage

    # Handle VP milestone (which occurs at months_to_vp)
    vp_milestone_percent = schedule_h_milestones_triggers["Vacant Possession (VP) with CCC"]
    spa_amount_vp = total_spa_price * (vp_milestone_percent / 100)
    loan_drawn_vp = spa_amount_vp * loan_percentage_decimal

    if cumulative_loan_drawn_at_milestone + loan_drawn_vp > total_loan_amount + 0.01:
        loan_drawn_vp = max(0, total_loan_amount - cumulative_loan_drawn_at_milestone)

    # Add VP loan drawn amount to the loan_drawdowns dictionary for months_to_vp
    loan_drawdowns[months_to_vp] = loan_drawdowns.get(months_to_vp, 0) + loan_drawn_vp

    # Also handle the post-VP percentages (Submission for Subdivision, Defect Liability Periods)
    # For simplicity, these are assumed to be "fully paid" by the loan at the time they are due.
    # We are primarily tracking the *loan balance* for interest calculation.
    # If these triggered *new* loan drawdowns, the logic would be more complex.

    for post_vp_milestone_key in ["Submission for Subdivision of Building", "6 Months After VP (Defect Liability Period - 1st Half)", "18 Months After VP (Defect Liability Period - 2nd Half)"]:
        if post_vp_milestone_key in schedule_h_milestones_triggers:
            percentage = schedule_h_milestones_triggers[post_vp_milestone_key]
            spa_amount_post_vp = total_spa_price * (percentage / 100)
            loan_drawn_post_vp = spa_amount_post_vp * loan_percentage_decimal

            # Ensure these draws are also capped by the total loan amount
            if cumulative_loan_drawn_at_milestone + loan_drawn_post_vp > total_loan_amount + 0.01:
                loan_drawn_post_vp = max(0, total_loan_amount - cumulative_loan_drawn_at_milestone)

            cumulative_loan_drawn_at_milestone += loan_drawn_post_vp

            # Estimate month for these post-VP triggers
            post_vp_estimated_month = 0
            if "Submission for Subdivision" in post_vp_milestone_key:
                post_vp_estimated_month = months_to_vp + 3
            elif "6 Months After VP" in post_vp_milestone_key:
                post_vp_estimated_month = months_to_vp + 6
            elif "18 Months After VP" in post_vp_milestone_key:
                post_vp_estimated_month = months_to_vp + 18

            if post_vp_estimated_month > 0:
                loan_drawdowns[post_vp_estimated_month] = loan_drawdowns.get(post_vp_estimated_month, 0) + loan_drawn_post_vp

    # --- Generate the Monthly Schedule ---
    monthly_schedule_data = []
    current_cumulative_loan_balance = 0 # This tracks the outstanding loan balance for interest calculation
    total_pre_vp_interest_paid = 0

    # The maximum month to simulate is VP month + total loan amortization months
    max_simulated_month = months_to_vp + total_loan_months

    # PHASE 1: Pre-VP Payments (Interest-only on drawn amount)
    for month in range(0, months_to_vp + 1): # Iterate up to and including VP month
        loan_draw_this_month = loan_drawdowns.get(month, 0)
        current_cumulative_loan_balance += loan_draw_this_month

        # Ensure cumulative loan balance doesn't exceed total_loan_amount
        if current_cumulative_loan_balance > total_loan_amount:
            current_cumulative_loan_balance = total_loan_amount

        interest_payment_this_month = current_cumulative_loan_balance * monthly_interest_rate
        total_pre_vp_interest_paid += interest_payment_this_month

        monthly_schedule_data.append({
            "Month": month,
            "Type": "Pre-VP",
            "Loan Drawn (RM)": loan_draw_this_month,
            "Cumulative Loan (RM)": current_cumulative_loan_balance,
            # Removed "Principal Payment (RM)" column
            "Interest Payment (RM)": interest_payment_this_month,
            "Total Monthly Payment (RM)": interest_payment_this_month,
            "Remaining Loan Balance (RM)": current_cumulative_loan_balance # Loan still drawn but not amortized
        })

    # PHASE 2: Post-VP Full Amortization Payments
    remaining_principal_for_amortization = total_loan_amount # The full loan amount is now outstanding
    total_post_vp_interest_paid = 0

    if total_loan_amount > 0 and total_loan_months > 0:
        if monthly_interest_rate > 0:
            monthly_full_repayment = -npf.pmt(monthly_interest_rate, total_loan_months, total_loan_amount)
        else:
            monthly_full_repayment = total_loan_amount / total_loan_months # Interest-free loan

        for month_offset in range(1, total_loan_months + 1):
            current_month = months_to_vp + month_offset

            if remaining_principal_for_amortization <= 0.01:
                break # Loan fully paid

            interest_portion = remaining_principal_for_amortization * monthly_interest_rate
            principal_portion = monthly_full_repayment - interest_portion

            # Adjust last payment to clear remaining balance exactly
            if month_offset == total_loan_months:
                principal_portion = remaining_principal_for_amortization
                monthly_full_repayment_actual = principal_portion + interest_portion # The actual last payment
            else:
                monthly_full_repayment_actual = monthly_full_repayment


            if principal_portion < 0:
                principal_portion = 0

            remaining_principal_for_amortization -= principal_portion
            if remaining_principal_for_amortization < 0:
                remaining_principal_for_amortization = 0

            total_post_vp_interest_paid += interest_portion

            monthly_schedule_data.append({
                "Month": current_month,
                "Type": "Post-VP",
                "Loan Drawn (RM)": 0.00,
                "Cumulative Loan (RM)": total_loan_amount,
                # Removed "Principal Payment (RM)" column
                "Interest Payment (RM)": interest_portion,
                "Total Monthly Payment (RM)": monthly_full_repayment_actual,
                "Remaining Loan Balance (RM)": remaining_principal_for_amortization
            })

    # Create DataFrame for display
    df_schedule = pd.DataFrame(monthly_schedule_data)

    print("\n--- Detailed Monthly Payment Schedule ---")
    # Using to_string with float_format for consistent decimal places and na_rep for clarity
    print(df_schedule.to_string(index=False, float_format="%.2f", na_rep='-'))

    print(f"\n--- Payment Summary ---")
    print(f"Total Pre-VP Interest Paid: RM {total_pre_vp_interest_paid:,.2f}")
    print(f"Total Post-VP Interest Paid: RM {total_post_vp_interest_paid:,.2f}")
    print(f"Grand Total Interest Paid: RM {(total_pre_vp_interest_paid + total_post_vp_interest_paid):,.2f}")
    print(f"Total Principal Paid (Matches Loan Amount): RM {total_loan_amount:,.2f}")
    print(f"Total Overall Payments: RM {(total_loan_amount + total_pre_vp_interest_paid + total_post_vp_interest_paid):,.2f}")
    print("-" * 50)


# --- User Inputs & Function Call ---
try:
    print("Welcome to the Integrated Progressive Payment Calculator (Schedule H)")
    print("This will show monthly payments from signing to loan completion.")
    total_spa_price = float(input("Enter Total SPA Price (e.g., 500000): RM "))
    months_to_vp = int(input("Enter Estimated Months to Vacant Possession (e.g., 36): "))
    loan_percentage = float(input("Enter Loan Amount as Percentage of SPA Price (e.g., 90 for 90%): "))
    annual_interest_rate = float(input("Enter Annual Interest Rate (e.g., 3.5 for 3.5%): "))
    loan_tenure_years = int(input("Enter Loan Tenure in Years (e.g., 30): "))

    integrated_progressive_payment_calculator(
        total_spa_price,
        months_to_vp,
        loan_percentage,
        annual_interest_rate,
        loan_tenure_years
    )

except ValueError as ve:
    print(f"\nInput Error: {ve}")
except Exception as e:
    print(f"\nAn unexpected error occurred: {e}")

Welcome to the Integrated Progressive Payment Calculator (Schedule H)
This will show monthly payments from signing to loan completion.
Enter Total SPA Price (e.g., 500000): RM 500000
Enter Estimated Months to Vacant Possession (e.g., 36): 36
Enter Loan Amount as Percentage of SPA Price (e.g., 90 for 90%): 90
Enter Annual Interest Rate (e.g., 3.5 for 3.5%): 4
Enter Loan Tenure in Years (e.g., 30): 35

--- Property & Loan Summary ---
Total SPA Price: RM 500,000.00
Estimated Months to VP: 36 months
Loan Percentage: 90.00%
Annual Interest Rate: 4.00%
Loan Tenure: 35 years (420 months)
Total Loan Amount: RM 450,000.00
--------------------------------------------------

--- Detailed Monthly Payment Schedule ---
 Month    Type  Loan Drawn (RM)  Cumulative Loan (RM)  Interest Payment (RM)  Total Monthly Payment (RM)  Remaining Loan Balance (RM)
     0  Pre-VP         45000.00              45000.00                 150.00                      150.00                     45000.00
     1  Pre-VP   